In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from joblib import dump,load
from sklearn.feature_selection import RFECV

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y=Y['SLME @ 5um']
names=df.columns
names=['SHAP value of '+x for x in names]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 57) ; y_train's shape is (390,)
X_test's shape is (98, 57) ; y_test's shape is (98,)


In [3]:
##gridsearch and cv select model##
param_grid={ 'learning_rate': [0.01,0.05,0.1,0.15,0.20],
              'n_estimators': [15,50,100,200],
              'max_depth': [4,5,6,7,8,9],
              'min_child_weight': [3,4,6,7],
              'subsample': [0.7,0.8,0.9,1],
              'colsample_bytree': [0.3,0.5,0.6,0.7],
              'gamma': [0,0.1,0.2,0.3,0.4],
                'reg_alpha': [0,0.1,0.2,0.3],
                'reg_lambda': [1,2,5]
            }

In [4]:
xgb = XGBRegressor()
model = GridSearchCV(estimator =xgb,scoring='neg_root_mean_squared_error', 
                     param_grid = param_grid,cv=5, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('SLME_13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 5 folds for each of 460800 candidates, totalling 2304000 fits


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0            0.015032      0.001650         0.003302        0.000401   
1            0.019354      0.002527         0.004901        0.000918   
2            0.019428      0.003953         0.004307        0.000680   
3            0.017841      0.000727         0.004298        0.000405   
4            0.016973      0.001090         0.003910        0.001325   
...               ...           ...              ...             ...   
460795       0.203510      0.015383         0.003710        0.000984   
460796       0.172845      0.008254         0.003805        0.000514   
460797       0.177459      0.006287         0.003302        0.000870   
460798       0.184563      0.011498         0.003229        0.000692   
460799       0.178097      0.003873         0.003312        0.000395   

       param_colsample_bytree param_gamma param_learning_rate param_max_depth  \
0                         0.3           0             